<a href="https://colab.research.google.com/github/Skc69/MEMT680-/blob/main/MEM_T680_Final_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
# Add your import statements here

# for basic operations
import numpy as np 
import pandas as pd 
import os
import sys

!pip install urllib3
import urllib.request
import time

# from urllib.request import urlopen
# from urllib.parse import urlencode

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# for modeling 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import torch
from torch.utils.data import DataLoader
from torch import nn

import plotly.express as px

from imblearn.over_sampling import SMOTE

# to avoid warnings
import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
# This is a tool I have provided you to help you download your file.

def download_file(url, filename):
    """
    A function that downloads the data file from a URL
    Parameters
    ----------
    url : string
        url where the file to download is located
    filename : string
        location where to save the file
    reporthook : function
        callback to display the download progress
    """
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename, reporthook)
        
def reporthook(count, block_size, total_size):
    """
    A function that displays the status and speed of the download
    """

    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration + 0.0001))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                     (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()

In [62]:
# You can download your file by typing your first name into the name block
# The name used is the first part of your first name as listed in BB learn
# If you have problems downloading the data please reach out to me

name = 'Sharod'
download_file(f'https://zenodo.org/record/7339649/files/data_{name}.npz?download=1','data.npz')

In [63]:
data = np.load('/content/data.npz')


In [64]:
training_feature=data['training_feat']
training_values=data['training_true']
validation_feature=data['validation_feat']

In [65]:
print(training_feature)
print(training_values)
print(validation_feature)

[[43.10694359 50.76156193  0.14872374 ... 10.0783307  40.51447868
  11.03899045]
 [24.75580839 19.84139653  0.09375188 ... 11.58820466  9.66105459
  16.09292735]
 [ 3.46623799  3.73336513  0.35150801 ...  2.54182479 62.1467037
  13.26933707]
 ...
 [29.17649777 33.19229449  0.30163132 ... 11.24660781 27.02571867
   3.315562  ]
 [30.58514331 63.0059261   1.03874858 ...  9.57315767 67.33687135
   3.50440809]
 [ 8.04169912 62.79639029  1.08765594 ...  7.61812899 65.59999968
   4.8771185 ]]
[[0.62649916 0.47504067 0.44176576]
 [0.61971988 0.50512314 0.69298578]
 [0.74123029 0.25361918 0.50402127]
 ...
 [0.32689598 0.64644768 0.30144564]
 [0.41800837 0.62838638 0.5584109 ]
 [0.439153   0.58602573 0.84111703]]
[[ 3.88610911  4.65685415  0.35829955 ... 12.86851755 19.4151537
  21.00803104]
 [ 3.88610911  4.65685415  0.35829955 ... 12.86851755 19.4151537
  21.00803104]
 [ 3.88610911  4.65685415  0.35829955 ... 12.86851755 19.4151537
  21.00803104]
 ...
 [ 3.88610911  4.65685415  0.35829955 ... 

In [66]:
#Regression Values, min-max scalar
mm_scaler = MinMaxScaler()
scaled_training_values = mm_scaler.fit_transform(training_values)   

#Training features, standard scaler
s_scaler =  StandardScaler()
scaled_training_feature = s_scaler.fit_transform(training_feature)

In [67]:
class Data():
  '''Dataset Class to store the samples and their corresponding labels, 
  and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
  '''

  def __init__(self, X: np.ndarray, y: np.ndarray, device = 'cuda') -> None:

    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    self.X = X.astype(np.float32)
    self.y = y.astype(np.float32)
    self.len = len(self.X)
  
  def __getitem__(self, index: int) -> tuple:
    return self.X[index], self.y[index]

  def __len__(self) -> int:
    return self.len

In [68]:
#Split train/test and put into Data object
X_train, X_test, y_train, y_test = train_test_split(scaled_training_feature, scaled_training_values, test_size = 0.33, random_state = 42)
training_data = Data(X_train, y_train)

#check results
print(training_data.X.shape, training_data.y.shape)

(67000, 30) (67000, 3)


In [69]:
Training_dataloader = DataLoader(training_data, batch_size = 64, shuffle=True)

In [70]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [71]:
class Neural_Network(nn.Module):
  ''' Regression Model
  ''' 

  # note, you can ignore the `:int` and `-> None` this is just more advanced doctring syntax
  def __init__(self, input_dim: int, hidden_dim: int, output_dim: int) -> None:
      '''The network has 4 layers
            - input layer
            - ReLu
            - hidden layer
            - ReLu
            - hidden layer
            - ReLu
            - output layer
      '''
      super(Neural_Network, self).__init__()
      self.flatten = nn.Flatten()
      self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
      # in this part you should intantiate each of the layer components
      # Type your code here
      

  def forward(self, x: torch.Tensor) -> torch.Tensor:
      # In this part you should build a model that returns the 3 outputs of the regression
      # Type your code here
      def forward(self, x: torch.Tensor) -> torch.Tensor:
      # In this part you should build a model that returns the 3 outputs of the regression
        x = self.flatten(x)
      preds = self.linear_relu_stack(x)
      
      return preds
      return x

In [72]:
# number of features (len of X cols)
input_dim = X_train.shape
# number of hidden layers set this to 50
hidden_layers = 50
# Add the number of output dimensions
output_dim = 3

In [73]:
# initiate the regression model
# make sure to put it on your GPU
model = Neural_Network(input_dim, hidden_layers, output_dim).to(device)
print(model)

# criterion to computes the loss between input and target
# Choose a good criteria
loss_fn = nn.MSELoss()
# optimizer that will be used to update weights and biases
# you can choose any optimizer. I would recommend ADAM.
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-5)
# This problem should not be hard to optimize. A good starting learning rate is 3e-5. 

Neural_Network(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [74]:
# start training
epochs = 20
# sets the number of epochs to train 20 should be sufficent.
# This should take about 5-10 minutes to train.
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
# Your code should go here
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: ignored